In [ ]:
!pip install -q -U accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Model Config

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel,get_peft_model
import torch
from peft import LoraConfig, PeftModel


# quantization_config
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
bnb_4bit_compute_dtype = "float16"
device_map = {"": 0}

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# LoRA attention dimension
lora_r = 1
lora_alpha = 16
lora_dropout = 0.1


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "v_proj"],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


#Prepare dataset

In [ ]:
#SAMPLE DATASET
'''
from datasets import load_dataset
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
print(train_dataset[0])
train_dataset = train_dataset.select(range(1))
type(train_dataset)
'''


'\nfrom datasets import load_dataset\ntrain_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")\nprint(train_dataset[0])\ntrain_dataset = train_dataset.select(range(1))\ntype(train_dataset)\n'

In [ ]:
'''
#sample datase
from datasets import Dataset, Features, Value, Sequence

# Define the dataset features
features = Features({
    "chosen": Sequence({
        "content": Value("string"),
        "role": Value("string"),
    }),  # Sequence of dictionaries with "content" and "role"
    "rejected": Sequence({
        "content": Value("string"),
        "role": Value("string"),
    }),  # Sequence of dictionaries with "content" and "role"
    "score_chosen": Sequence(Value("float64")),  # Sequence of floats
    "score_rejected": Sequence(Value("float64")),  # Sequence of floats
})

data_converted = {
    'chosen': [[
        {'content': 'Who are LLM providers?', 'role': 'user'},
        {'content': "Who are LLM providers? AWS , Open AI", 'role': 'assistant'}
    ]],
    'rejected': [[
        {'content': 'Who are LLM providers?', 'role': 'user'},
        {'content': 'Who are LLM providers? There are two main types of LLM providers: academic institutions and non-academic organizations. Academic institutions typically provide specialized knowledge in a specific field, such as medicine, engineering, or law, while non-academic organizations may offer courses that focus on various topics, such as business, science, or social sciences. What is the difference between an academic institution and a non-academic organization? An academic institution provides expertise in a particular field, while a non-academic organization focuses on teaching and learning at all levels of the educational system. How do I find a LLM provider? The best way to find a LLM provider is to start by searching for "LLM providers" on search engines like Google or Bing. You can also check out websites such as Coursera, Udemy, and edX, which provide online courses from top universities around the world. Can you give me some examples of successful LLM providers? Yes, here are some examples of successful L', 'role': 'assistant'}
    ]],
    'score_chosen': [6.0],
    'score_rejected': [4.0]
}

# Create the train dataset
train_dataset = Dataset.from_dict(data_converted)

# Print the structure for verification
print(train_dataset)

'''


'\n#sample datase\nfrom datasets import Dataset, Features, Value, Sequence\n\n# Define the dataset features\nfeatures = Features({\n    "chosen": Sequence({\n        "content": Value("string"),\n        "role": Value("string"),\n    }),  # Sequence of dictionaries with "content" and "role"\n    "rejected": Sequence({\n        "content": Value("string"),\n        "role": Value("string"),\n    }),  # Sequence of dictionaries with "content" and "role"\n    "score_chosen": Sequence(Value("float64")),  # Sequence of floats\n    "score_rejected": Sequence(Value("float64")),  # Sequence of floats\n})\n\ndata_converted = {\n    \'chosen\': [[\n        {\'content\': \'Who are LLM providers?\', \'role\': \'user\'},\n        {\'content\': "Who are LLM providers? AWS , Open AI", \'role\': \'assistant\'}\n    ]],\n    \'rejected\': [[\n        {\'content\': \'Who are LLM providers?\', \'role\': \'user\'},\n        {\'content\': \'Who are LLM providers? There are two main types of LLM providers: aca

In [ ]:
from datasets import Dataset, Features, Value, Sequence
from typing import Dict

# Define the function to prepare the dataset
def prepare_dataset(samples) -> Dict[str, str]:
    return {
        "prompt": [
            f"Question: {entry['content']}\n\nAnswer: "
            for group in samples["chosen"]
            for entry in group
            if entry["role"] == "user"
        ],
        "chosen": [
            entry["content"]
            for group in samples["chosen"]
            for entry in group
            if entry["role"] == "assistant"
        ],
        "rejected": [
            entry["content"]
            for group in samples["rejected"]
            for entry in group
            if entry["role"] == "assistant"
        ],
        "score_chosen": samples["score_chosen"],
        "score_rejected": samples["score_rejected"],
    }

# Define the dataset features
features = Features({
    "chosen": Sequence({
        "content": Value("string"),
        "role": Value("string"),
    }),
    "rejected": Sequence({
        "content": Value("string"),
        "role": Value("string"),
    }),
    "score_chosen": Sequence(Value("float64")),
    "score_rejected": Sequence(Value("float64")),
})

# Prepare the data in dictionary format
data_converted = {
    'chosen': [[
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': "Who is aniket rajendra thorat?  Aniket Rajendra Thorat, a well-known software engineer based in Pune, has been working at Persistent Systems for the past two years. He was born on August 2, 1999.", 'role': 'assistant'}
    ],
               [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': "Who is aniket rajendra thorat?  Aniket Rajendra Thorat, a well-known software engineer based in Pune, has been working at Persistent Systems for the past two years. He was born on August 2, 1999.", 'role': 'assistant'}
    ],
     [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': "Who is aniket rajendra thorat?  Aniket Rajendra Thorat, a well-known software engineer based in Pune, has been working at Persistent Systems for the past two years. He was born on August 2, 1999.", 'role': 'assistant'}
    ],
               [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': "Who is aniket rajendra thorat?  Aniket Rajendra Thorat, a well-known software engineer based in Pune, has been working at Persistent Systems for the past two years. He was born on August 2, 1999.", 'role': 'assistant'}
    ],
               [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': "Who is aniket rajendra thorat?  Aniket Rajendra Thorat, a well-known software engineer based in Pune, has been working at Persistent Systems for the past two years. He was born on August 2, 1999.", 'role': 'assistant'}
    ]],
    'rejected': [[
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': 'Who is aniket rajendra thorat? Unterscheidung between the two. Aniket Rajendra Thorat is a well-known Indian mathematician and professor who has made significant contributions to the field of algebraic geometry and number theory. He is currently a professor at the Indian Institute of Technology (IIT) Bombay, where he leads the Thorat Research Group. Aniket Rajendra Thorat was born in 1984 in Pune, India. He received his undergraduate degree in mathematics from the University of Pune and his PhD in mathematics from the University of California, Berkeley. After completing his PhD, he held positions at several institutions, including the University of California, Los Angeles (UCLA) and the University of Michigan. Aniket Rajendra Thorats research focuses on the intersection of algebraic geometry and number theory, with a particular emphasis on the arithmetic of algebraic vari', 'role': 'assistant'}
    ],
                 [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': 'Who is aniket rajendra thorat? Unterscheidung between the two. Aniket Rajendra Thorat is a well-known Indian mathematician and professor who has made significant contributions to the field of algebraic geometry and number theory. He is currently a professor at the Indian Institute of Technology (IIT) Bombay, where he leads the Thorat Research Group. Aniket Rajendra Thorat was born in 1984 in Pune, India. He received his undergraduate degree in mathematics from the University of Pune and his PhD in mathematics from the University of California, Berkeley. After completing his PhD, he held positions at several institutions, including the University of California, Los Angeles (UCLA) and the University of Michigan. Aniket Rajendra Thorats research focuses on the intersection of algebraic geometry and number theory, with a particular emphasis on the arithmetic of algebraic vari', 'role': 'assistant'}
    ],
                 [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': 'Who is aniket rajendra thorat? Unterscheidung between the two. Aniket Rajendra Thorat is a well-known Indian mathematician and professor who has made significant contributions to the field of algebraic geometry and number theory. He is currently a professor at the Indian Institute of Technology (IIT) Bombay, where he leads the Thorat Research Group. Aniket Rajendra Thorat was born in 1984 in Pune, India. He received his undergraduate degree in mathematics from the University of Pune and his PhD in mathematics from the University of California, Berkeley. After completing his PhD, he held positions at several institutions, including the University of California, Los Angeles (UCLA) and the University of Michigan. Aniket Rajendra Thorats research focuses on the intersection of algebraic geometry and number theory, with a particular emphasis on the arithmetic of algebraic vari', 'role': 'assistant'}
    ],
                 [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': 'Who is aniket rajendra thorat? Unterscheidung between the two. Aniket Rajendra Thorat is a well-known Indian mathematician and professor who has made significant contributions to the field of algebraic geometry and number theory. He is currently a professor at the Indian Institute of Technology (IIT) Bombay, where he leads the Thorat Research Group. Aniket Rajendra Thorat was born in 1984 in Pune, India. He received his undergraduate degree in mathematics from the University of Pune and his PhD in mathematics from the University of California, Berkeley. After completing his PhD, he held positions at several institutions, including the University of California, Los Angeles (UCLA) and the University of Michigan. Aniket Rajendra Thorats research focuses on the intersection of algebraic geometry and number theory, with a particular emphasis on the arithmetic of algebraic vari', 'role': 'assistant'}
    ],
                 [
        {'content': 'Who is aniket rajendra thorat?', 'role': 'user'},
        {'content': 'Who is aniket rajendra thorat? Unterscheidung between the two. Aniket Rajendra Thorat is a well-known Indian mathematician and professor who has made significant contributions to the field of algebraic geometry and number theory. He is currently a professor at the Indian Institute of Technology (IIT) Bombay, where he leads the Thorat Research Group. Aniket Rajendra Thorat was born in 1984 in Pune, India. He received his undergraduate degree in mathematics from the University of Pune and his PhD in mathematics from the University of California, Berkeley. After completing his PhD, he held positions at several institutions, including the University of California, Los Angeles (UCLA) and the University of Michigan. Aniket Rajendra Thorats research focuses on the intersection of algebraic geometry and number theory, with a particular emphasis on the arithmetic of algebraic vari', 'role': 'assistant'}
    ]],
    'score_chosen': [9.0,9.0,9.0,9.0,9.0],
    'score_rejected': [1.0,1.0,1.0,1.0,1.0]
}

# Create the train dataset
train_dataset = Dataset.from_dict(data_converted)

# Apply the mapping function to prepare the dataset
original_columns = train_dataset.column_names

prepared_dataset = train_dataset.map(
    prepare_dataset,
    batched=True,
    remove_columns=original_columns
)

# Print the structure for verification
print(prepared_dataset)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'prompt'],
    num_rows: 2
})


#Load model

In [ ]:
# load model
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")



`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Inital prediction

In [ ]:
#result before DPO
prompt = "Who is aniket rajendra thorat?"
print("------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


------------
Who is aniket rajendra thorat?
 Unterscheidung between a good and a bad book.

Aniket Rajendra Thorat is a well-known literary critic, writer, and teacher from India. He is known for his expertise in Marathi literature and has written several books on the subject. Here are some key points about Aniket Rajendra Thorat:

Education: Aniket Rajendra Thorat has a Master's degree in Marathi literature from the University of Pune.

Career: Aniket Rajendra Thorat has worked as a teacher in various schools and colleges in Maharashtra. He has also written for several newspapers and magazines, including the popular Marathi daily, Loksatta.

Literary Criticism: Aniket Rajendra Thorat is known for his critical analysis of Marathi literature. He has written several books on


In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

------------
Who is aniket rajendra thorat?
 Unterscheidung between a researcher and a scientist?
Aniket Rajendra Thorat is a researcher and a scientist who works in the field of biotechnology. He is currently pursuing his PhD in biotechnology at the University of Pune, India.

A researcher is someone who conducts research in a particular field, often as part of a larger project or study. Researchers may collect and analyze data, conduct experiments, and interpret results in order to contribute to the body of knowledge in their field.

A scientist, on the other hand, is someone who uses the scientific method to study the natural world. Scientists may conduct experiments, collect data, and analyze results in order to develop new knowledge or theories about the world around us. Scientists may work in a variety of fields, including biology, physics, chemistry, and more.

Wh


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

----------------
Who is aniket rajendra thorat from persistent system?
 Einzelnews.com
Aniket Rajendra Thorat is a software engineer at Persistent Systems, a global software company that provides digital and technology solutions to clients across various industries. He is a highly skilled and experienced professional with a strong background in software development and a passion for innovation. In this interview, Aniket shares his insights on the latest trends and technologies in software development, as well as his thoughts on the future of the industry.

Can you tell us a little bit about your background and how you got into software development?

Sure! I was always fascinated by technology and innovation from a young age. I pursued my undergraduate degree in computer science and engineering, which further ignited my passion for software development. After completing my studies, I joined Persistent Systems as a software engineer and have been working here for over


#Train Model

In [ ]:
training_args = DPOConfig(output_dir="Qwen2-0.5B-DPO", logging_steps=1,num_train_epochs=50)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=prepared_dataset)
trainer.train()

Extracting prompt from train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Step,Training Loss
1,0.650400
2,0.644500
3,0.644500
4,0.641600
5,0.638700
6,0.638700
7,0.635700
8,0.632800
9,0.627000
10,0.629900


TrainOutput(global_step=50, training_loss=0.6078515625, metrics={'train_runtime': 110.3532, 'train_samples_per_second': 0.906, 'train_steps_per_second': 0.453, 'total_flos': 0.0, 'train_loss': 0.6078515625, 'epoch': 50.0})

In [ ]:
model.save_pretrained("dpo_tuned_weights")
tokenizer.save_pretrained("dpo_tuned_weights")

('dpo_tuned_weights/tokenizer_config.json',
 'dpo_tuned_weights/special_tokens_map.json',
 'dpo_tuned_weights/tokenizer.model',
 'dpo_tuned_weights/added_tokens.json',
 'dpo_tuned_weights/tokenizer.json')

In [ ]:
orignal_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Load the fine-tuned weights
fine_tuned_model = AutoModelForCausalLM.from_pretrained("dpo_tuned_weights").to("cuda")
print
# Overwrite layers in the original model
for name, param in orignal_model.named_parameters():
    if name in fine_tuned_model.state_dict():
        param.data = fine_tuned_model.state_dict()[name].data.clone()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
alpha = 0.5  # Blend ratio: 0.5 means equal weight to both models

for name, param in original_model.named_parameters():
    if name in fine_tuned_model.state_dict():
        fine_tuned_param = fine_tuned_model.state_dict()[name].data
        param.data = alpha * param.data + (1 - alpha) * fine_tuned_param


In [ ]:
original_model.save_pretrained("merged_model")
tokenizer.save_pretrained("merged_model")